In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [2]:
data = pd.read_stata('C:\\Users\\bayle\\Documents\\HW4.dta')
display(data)

,person_ID,year,log_wage
0,4.0,1982,9.615806
1,5.0,1974,9.479604
2,5.0,1975,9.384294
3,5.0,1976,7.228388
4,5.0,1977,9.428673
...,...,...,...
33774,35355.0,1975,9.026418
33775,35355.0,1976,8.780787
33776,35355.0,1977,9.086702
33777,35356.0,1975,8.348064


In [3]:
variables_list = data.columns.tolist()
print(variables_list)

['person_ID', 'year', 'log_wage']


In [4]:
variance_by_year = data.groupby('year')['log_wage'].var()
covariance_across_years = data.pivot_table(index='person_ID', columns='year', values='log_wage').cov()
#display(covariance_across_years)
data_moments = covariance_across_years.reset_index()
data_moments = data_moments.iloc[:,1:].values
data_moments = np.tril(data_moments)
data_moments = data_moments.reshape(-1,1)
data_moments = data_moments[data_moments!=0].reshape(-1,1)
#display(data_moments)

## VVD simulation

In [5]:
def simulate_income_data(p, num_individuals, num_periods):

    var_alpha, rho, var_eta, var_epsilon = p

    np.random.seed(4) # VVD
    
    # Simulate individual-specific fixed effects
    individual_fixed_effects = np.random.normal(scale=np.sqrt(var_alpha), size=num_individuals)
    
    # Initialize DataFrame to store simulated income data
    simulated_data = pd.DataFrame(index=range(num_individuals), columns=range(num_periods))
    
    # Simulate income data for each individual
    for i in range(num_individuals):

        # Initialise the autoregressive process
        z_process = 0
        
        # Generate income data for each period
        for t in range(num_periods):
            
            # Simulate shock to the autoregressive process
            eta = np.random.normal(scale=np.sqrt(var_eta))
            
            # Update autoregressive process
            z_process = rho * z_process + eta
            
            # Simulate income for the current period
            income = individual_fixed_effects[i] + z_process + np.random.normal(scale=np.sqrt(var_epsilon))
            
            # Store simulated income in DataFrame
            simulated_data.at[i, t] = income
    
    cov_matrix = simulated_data.cov()

    simulated_moments = cov_matrix.values

    simulated_moments = np.tril(simulated_moments)

    simulated_moments = simulated_moments.reshape(-1,1)

    simulated_moments = simulated_moments[simulated_moments!=0].reshape(-1,1) 

    return simulated_moments

In [6]:
# Simulate income data
params = [0.1,0.8,0.1,0.1]
simulated_moments = simulate_income_data(params, num_individuals=1000, num_periods=19)

In [7]:
def metric(p, data_moments, num_individuals, num_periods):

    simulated_moments = simulate_income_data(p, num_individuals, num_periods)

    distance = simulated_moments - data_moments

    dist = np.dot(distance.T, distance)
    
    return dist

In [7]:
dist = metric(params, data_moments, 1000, 19)
print(dist)

[[2.21436166]]


In [8]:
def SMM(p, num_individuals, num_periods, data_moments):

    bounds = [(0, 1), (0, 1), (0, 1), (0, 1)]

    result = minimize(metric, p, args=(data_moments, num_individuals, num_periods), method='Nelder-Mead', bounds=bounds)

    matched_coefficients = result.x

    return matched_coefficients

In [9]:
initial_params=[0.5,0.9,0.5,0.5]
initial_params_array = np.array(initial_params)

In [244]:
coeffs = SMM(initial_params_array, 1000, 19, data_moments)
print(coeffs)

[0.13803823 0.91605702 0.05730137 0.20404534]


## Now that the code runs, let's run it using 11 different values for the seed (1 of which is the seed used above)

In [10]:
def simulate_income_data_seeded(p, num_individuals, num_periods, seed):

    var_alpha, rho, var_eta, var_epsilon = p

    np.random.seed(seed) # VVD
    
    # Simulate individual-specific fixed effects
    individual_fixed_effects = np.random.normal(scale=np.sqrt(var_alpha), size=num_individuals)
    
    # Initialize DataFrame to store simulated income data
    simulated_data = pd.DataFrame(index=range(num_individuals), columns=range(num_periods))
    
    # Simulate income data for each individual
    for i in range(num_individuals):

        # Initialise the autoregressive process
        z_process = 0
        
        # Generate income data for each period
        for t in range(num_periods):
            
            # Simulate shock to the autoregressive process
            eta = np.random.normal(scale=np.sqrt(var_eta))
            
            # Update autoregressive process
            z_process = rho * z_process + eta
            
            # Simulate income for the current period
            income = individual_fixed_effects[i] + z_process + np.random.normal(scale=np.sqrt(var_epsilon))
            
            # Store simulated income in DataFrame
            simulated_data.at[i, t] = income
    
    cov_matrix = simulated_data.cov()

    simulated_moments = cov_matrix.values

    simulated_moments = np.tril(simulated_moments)

    simulated_moments = simulated_moments.reshape(-1,1)

    simulated_moments = simulated_moments[simulated_moments!=0].reshape(-1,1) 

    return simulated_moments

In [11]:
def metric_seeded(p, data_moments, num_individuals, num_periods, seed):

    simulated_moments = simulate_income_data_seeded(p, num_individuals, num_periods, seed)

    distance = simulated_moments - data_moments

    dist = np.dot(distance.T, distance)
    
    return dist

In [12]:
def SMM_seeded(p, num_individuals, num_periods, data_moments, seed):

    bounds = [(0, 1), (0, 1), (0, 1), (0, 1)]

    result = minimize(metric_seeded, p, args=(data_moments, num_individuals, num_periods, seed), method='Nelder-Mead', bounds=bounds)

    matched_coefficients = result.x

    return matched_coefficients

In [59]:
coeffs_1 = SMM_seeded(initial_params_array, 1000, 19, data_moments, 26)
print(coeffs_1)

[0.14918316 0.90575493 0.05983584 0.2040758 ]


In [56]:
seeds = [ 1, 3, 4, 5, 8, 9, 10, 11, 20, 26, 66]

In [57]:
def seeds_loop(s, p, num_inds, num_pds, data):

    seeds = np.array(s)

    values = np.zeros(shape=(11, 4))

    for i in range(len(seeds)):

        values[i,:] = SMM_seeded(p, num_inds, num_pds, data, seeds[i])

    return values

In [58]:
vals = seeds_loop(seeds, params, 1000, 19, data_moments)
print(vals)

[[0.13899356 0.89841806 0.06233006 0.19873708]
 [0.15572182 0.87074566 0.07022673 0.18661908]
 [0.13805628 0.91609638 0.05726215 0.20407815]
 [0.12391966 0.88979949 0.06694634 0.1860586 ]
 [0.11830516 0.89061022 0.06577114 0.1942395 ]
 [0.13508864 0.87550667 0.06613267 0.19380673]
 [0.16287631 0.86786999 0.07039855 0.18847897]
 [0.12760293 0.87042895 0.06840463 0.18546852]
 [0.12284117 0.90647172 0.05680831 0.20327515]
 [0.14923021 0.90567062 0.05987165 0.20406751]
 [0.15563031 0.87970576 0.06778917 0.18906245]]


## Work in progress from here on out.... trying to get weighting matrix working correctly

In [35]:
newer_variance_by_year = data.groupby('year')['log_wage'].var()
newer_covariance_across_years = data.pivot_table(index='person_ID', columns='year', values='log_wage').cov()
newer_data_moments_raw = covariance_across_years.reset_index()
newer_data_moments_raw = newer_data_moments_raw.iloc[:,1:].values
newer_data_moments_raw = np.tril(newer_data_moments_raw)
#display(newer_data_moments_raw)
newer_weighting_matrix=np.zeros(shape=(190,190))
newer_weighting_matrix[np.diag_indices_from(newer_weighting_matrix)] = newer_data_moments_raw[newer_data_moments_raw != 0]
#display(weighting_matrix.shape)
newer_weighting_matrix = np.where(newer_weighting_matrix != 0, 1. / newer_weighting_matrix, 0)
#display(newer_weighting_matrix)

C:\Users\bayle\AppData\Local\Temp\ipykernel_15172\4137456590.py:10: RuntimeWarning: divide by zero encountered in divide
  newer_weighting_matrix = np.where(newer_weighting_matrix != 0, 1. / newer_weighting_matrix, 0)


In [39]:
def simulate_income_data_newer(p, num_individuals, num_periods, seed):

    var_alpha, rho, var_eta, var_epsilon = p

    np.random.seed(seed) # VVD
    
    # Simulate individual-specific fixed effects
    individual_fixed_effects = np.random.normal(scale=np.sqrt(var_alpha), size=num_individuals)
    
    # Initialize DataFrame to store simulated income data
    simulated_data = pd.DataFrame(index=range(num_individuals), columns=range(num_periods))
    
    # Simulate income data for each individual
    for i in range(num_individuals):

        # Initialise the autoregressive process
        z_process = 0
        
        # Generate income data for each period
        for t in range(num_periods):
            
            # Simulate shock to the autoregressive process
            eta = np.random.normal(scale=np.sqrt(var_eta))
            
            # Update autoregressive process
            z_process = rho * z_process + eta
            
            # Simulate income for the current period
            income = individual_fixed_effects[i] + z_process + np.random.normal(scale=np.sqrt(var_epsilon))
            
            # Store simulated income in DataFrame
            simulated_data.at[i, t] = income
    
    cov_matrix = simulated_data.cov()

    simulated_moments = cov_matrix.values

    simulated_moments = np.tril(simulated_moments)

    simulated_moments = simulated_moments.reshape(-1,1)

    simulated_moments = simulated_moments[simulated_moments!=0].reshape(-1,1) 

    return simulated_moments

In [49]:
def metric_newer(p, data_moments, num_individuals, num_periods, seed, newer_weighting_matrix):

    simulated_moments = simulate_income_data_seeded(p, num_individuals, num_periods, seed)

    distance = simulated_moments - data_moments

    out = np.matmul(distance.T, newer_weighting_matrix)

    dist = np.matmul(out, distance)
    
    return dist

In [50]:
def SMM_newer(p, num_individuals, num_periods, data_moments, seed, newer_weighting_matrix):

    bounds = [(0, 1), (0, 1), (0, 1), (0, 1)]

    result = minimize(metric_newer, p, args=(data_moments, num_individuals, num_periods, seed, newer_weighting_matrix), method='Nelder-Mead', bounds=bounds)

    matched_coefficients = result.x

    return matched_coefficients

In [53]:
coeffs_newer = SMM_newer(initial_params_array, 1000, 19, data_moments, 66, newer_weighting_matrix)
print(coeffs_newer)

[0.15684932 0.87026719 0.07139512 0.16982923]


In [60]:
def seeds_loop_newer(s, p, num_inds, num_pds, data, newer_weighting_matrix):

    seeds = np.array(s)

    vals = np.zeros(shape=(11, 4))

    for i in range(len(seeds)):

        vals[i,:] = SMM_newer(p, num_inds, num_pds, data, seeds[i], newer_weighting_matrix)

    return vals

In [61]:
vals_newer = seeds_loop_newer(seeds, params, 1000, 19, data_moments, newer_weighting_matrix)
print(vals_newer)

[[0.1419799  0.89097232 0.06435482 0.18272868]
 [0.15446919 0.8683401  0.0712595  0.17374009]
 [0.14011806 0.91310592 0.05734853 0.19487085]
 [0.12234794 0.88782858 0.06804198 0.17524335]
 [0.12067007 0.8839199  0.06779949 0.1802715 ]
 [0.1338009  0.87303392 0.06721523 0.17912126]
 [0.1608566  0.86527094 0.07200634 0.17066166]
 [0.12822756 0.86511209 0.07029822 0.17198074]
 [0.1240573  0.90327049 0.05710609 0.19052547]
 [0.15215585 0.90115104 0.06034008 0.19278856]
 [0.15683588 0.87027028 0.07138808 0.16986306]]


In [9]:
avg_params_id = [0.138933277,0.888302138,0.064721945,0.193990158]
avg_params_id = np.array(avg_params_id)
avg_params_weight = [0.139592659, 0.883843235, 0.066105305, 0.180163202]
avg_params_weight = np.array(avg_params_weight)

In [23]:
def simulate_income_data_id_w(p, num_individuals, num_periods, seed):

    var_alpha, rho, var_eta, var_epsilon = p

    np.random.seed(seed) # VVD
    
    # Simulate individual-specific fixed effects
    individual_fixed_effects = np.random.normal(scale=np.sqrt(var_alpha), size=num_individuals)
    
    # Initialize DataFrame to store simulated income data
    simulated_data = pd.DataFrame(index=range(num_individuals), columns=range(num_periods))
    
    # Simulate income data for each individual
    for i in range(num_individuals):

        # Initialise the autoregressive process
        z_process = 0
        
        # Generate income data for each period
        for t in range(num_periods):
            
            # Simulate shock to the autoregressive process
            eta = np.random.normal(scale=np.sqrt(var_eta))
            
            # Update autoregressive process
            z_process = rho * z_process + eta
            
            # Simulate income for the current period
            income = individual_fixed_effects[i] + z_process + np.random.normal(scale=np.sqrt(var_epsilon))
            
            # Store simulated income in DataFrame
            simulated_data.at[i, t] = income
    
    cov_matrix = simulated_data.cov()

    #simulated_moments = cov_matrix.values 

    return cov_matrix

In [24]:
final_out_id = simulate_income_data_id_w(avg_params_id, 100000, 19, 2)
display(final_out_id)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.400876,0.198290,0.191060,0.186109,0.180254,0.175760,0.171207,0.169023,0.166475,0.161957,0.160155,0.158687,0.155613,0.153760,0.152331,0.150840,0.150790,0.148561,0.147912
1,0.198290,0.448533,0.243593,0.232049,0.221193,0.211292,0.202975,0.198303,0.190051,0.183028,0.179630,0.177805,0.171850,0.169657,0.164529,0.161413,0.160257,0.157270,0.155064
2,0.191060,0.243593,0.490475,0.281561,0.264016,0.250454,0.233983,0.224948,0.216501,0.205966,0.198246,0.192791,0.185636,0.181555,0.176811,0.172199,0.168907,0.165611,0.163058
3,0.186109,0.232049,0.281561,0.523315,0.306299,0.290670,0.270913,0.256419,0.245559,0.230942,0.223874,0.214061,0.204827,0.197201,0.193469,0.186089,0.180652,0.175014,0.172904
4,0.180254,0.221193,0.264016,0.306299,0.545101,0.328009,0.305515,0.287361,0.272326,0.255097,0.244832,0.233064,0.220612,0.214684,0.206698,0.197965,0.192692,0.184983,0.179303
5,0.175760,0.211292,0.250454,0.290670,0.328009,0.568909,0.345068,0.322816,0.302309,0.284108,0.269308,0.255662,0.241085,0.232640,0.221105,0.212989,0.203899,0.196364,0.190088
6,0.171207,0.202975,0.233983,0.270913,0.305515,0.345068,0.578785,0.356780,0.334582,0.310633,0.292296,0.275330,0.258431,0.249718,0.234039,0.222745,0.213763,0.203173,0.197504
7,0.169023,0.198303,0.224948,0.256419,0.287361,0.322816,0.356780,0.591698,0.369606,0.341885,0.320142,0.301020,0.281925,0.268752,0.254060,0.240569,0.230223,0.218398,0.211115
8,0.166475,0.190051,0.216501,0.245559,0.272326,0.302309,0.334582,0.369606,0.604671,0.377744,0.350484,0.328798,0.307320,0.290292,0.272557,0.257959,0.244836,0.232798,0.222515
9,0.161957,0.183028,0.205966,0.230942,0.255097,0.284108,0.310633,0.341885,0.377744,0.610515,0.384290,0.358149,0.333677,0.314772,0.294732,0.277081,0.264168,0.247838,0.237889


In [25]:
final_out_w = simulate_income_data_id_w(avg_params_weight, 100000, 19, 2)
display(final_out_w)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.389047,0.199848,0.192281,0.187032,0.180940,0.176273,0.171612,0.169250,0.166595,0.162071,0.160190,0.158703,0.155668,0.153810,0.152387,0.150924,0.150885,0.148743,0.148091
1,0.199848,0.437371,0.245441,0.233334,0.221973,0.211720,0.203129,0.198110,0.189863,0.182752,0.179111,0.177157,0.171329,0.169050,0.164028,0.160959,0.159775,0.156881,0.154745
2,0.192281,0.245441,0.479365,0.283239,0.264953,0.250805,0.234045,0.224586,0.215878,0.205159,0.197323,0.191747,0.184644,0.180470,0.175777,0.171201,0.167976,0.164754,0.162220
3,0.187032,0.233334,0.283239,0.511874,0.307684,0.291075,0.270815,0.255767,0.244488,0.229701,0.222257,0.212418,0.203176,0.195578,0.191715,0.184443,0.179081,0.173538,0.171398
4,0.180940,0.221973,0.264953,0.307684,0.533167,0.328840,0.305493,0.286630,0.271074,0.253541,0.242844,0.230947,0.218520,0.212417,0.204441,0.195787,0.190585,0.183013,0.177383
5,0.176273,0.211720,0.250805,0.291075,0.328840,0.556279,0.345288,0.322106,0.301055,0.282274,0.267043,0.253124,0.238523,0.229891,0.218406,0.210207,0.201315,0.193887,0.187656
6,0.171612,0.203129,0.234045,0.270815,0.305493,0.345288,0.565790,0.356486,0.333348,0.308785,0.289921,0.272658,0.255639,0.246569,0.231030,0.219740,0.210840,0.200362,0.194675
7,0.169250,0.198110,0.224586,0.255767,0.286630,0.322106,0.356486,0.577986,0.368667,0.340145,0.317664,0.298100,0.278737,0.265305,0.250523,0.237013,0.226705,0.215050,0.207768
8,0.166595,0.189863,0.215878,0.244488,0.271074,0.301055,0.333348,0.368667,0.590375,0.376249,0.348198,0.325813,0.303963,0.286645,0.268782,0.254088,0.241050,0.229005,0.218872
9,0.162071,0.182752,0.205159,0.229701,0.253541,0.282274,0.308785,0.340145,0.376249,0.595718,0.382319,0.355370,0.330341,0.310999,0.290740,0.272929,0.259911,0.243710,0.233793
